## Instructions for Using the Script and Data

__Data Location__: /data/sjw6257/xDTD_database/zip_files/tables (`.tar.gz` file of data)

__Description__: 
This script is to conduct schema analysis for each model's Node. The data for each model is stored in KG databases (SQLite). There are three models: ExplainableDTD_v1.3_KG2.8.0.1, ExplainableDTD_v1.0_KG2.8.3, ExplainableDTD_v1.0_KG2.8.6. Extract `NODE_MAPPING_TABLE` in CSV file format from the databases in order to conduct the following analysis.

Please refer to the code below as example for the schema analysis:

In [ ]:
# Set working directory
# Copy and transfer the compressed data file (tar.gz) to the working directory before starting

import os
os.chdir('/home/grads/sjw6257/xDTD/xDTD_analysis')

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats

### Extract Node Representative ID('id' column)

In [ ]:
df_2801 = pd.read_csv('KG2.8.0.1_NodeMapping.csv', dtype='unicode', low_memory=False)
df_2801 = df_2801[['id','name']] 
#df_2801

In [ ]:
df_283 = pd.read_csv('KG2.8.3_NodeMapping.csv', dtype='unicode', low_memory=False)
df_283 = df_283[['id','name']]
#df_283

In [ ]:
df_286 = pd.read_csv('KG2.8.6_NodeMapping.csv', dtype='unicode', low_memory=False)
df_286 = df_286[['id','name']]
#df_286

### Model v2.8.0.1 versus Model v2.8.3 Comparison

In [ ]:
df_2801_intr_283 = pd.merge(df_2801,df_283, how='inner', on=['id'],suffixes=('_2801','_283')) 
df_2801_intr_283 

In [ ]:
# How much of the intersect match in respect to each database

percent_match_1 = (len(df_2801_intr_283[['id']]) / len(df_2801[['id']])) * 100
percent_match_2 = len(df_2801_intr_283[['id']])/ len(df_283[['id']]) *100
print(f"\nPercentage of IDs in df_2801 that match: {percent_match_1:.2f}%")
print(f"\nPercentage of IDs in df_283 that match: {percent_match_2:.2f}%")

### Model v2.8.0.1 versus Model v2.8.6 Comparison

In [ ]:
df_2801_intr_286 = pd.merge(df_2801,df_286, how='inner', on=['id'], suffixes=('_2801','_286')) 
df_2801_intr_286

In [ ]:
# How much of the intersect match in respect to each dataframes

percent_match_1 = (len(df_2801_intr_286[['id']]) / len(df_2801[['id']])) * 100
percent_match_2 = len(df_2801_intr_286[['id']])/ len(df_286[['id']]) *100
print(f"\nPercentage of IDs in df_2801 that match: {percent_match_1:.2f}%")
print(f"\nPercentage of IDs in df_286 that match: {percent_match_2:.2f}%")

### Model v2.8.3 versus Model v2.8.6 Comparison

In [ ]:
df_283_intr_286 = pd.merge(df_283,df_286, how='inner', on=['id'],suffixes=('_283','_286')) 
df_283_intr_286 

In [ ]:
# How much of the intersect match in respect to each dataframes

percent_match_1 = (len(df_283_intr_286[['id']]) / len(df_283[['id']])) * 100
percent_match_2 = len(df_283_intr_286[['id']])/ len(df_286[['id']]) *100
print(f"\nPercentage of IDs in df_283 that match: {percent_match_1:.2f}%")
print(f"\nPercentage of IDs in df_286 that match: {percent_match_2:.2f}%")

### Drug-Disease Pairs present in ALL three models

In [ ]:
# Looking at representative id's that are present in ALL THREE KG2.8.0.1, KG2.8.3_refresh, and KG2.8.6

df = df_2801.merge(df_283, on=['id']).merge(df_286, on=['id'], suffixes=('_2801','_286','_283'))
df.columns = ['id','2801', '283', '286'] # rename column

df_all = df[['id']]
df_all

In [ ]:
dataframes = {'df_2801': df_2801, 'df_283': df_283, 'df_286': df_286}
for name, df in dataframes.items():
    percent_match = len(df_all) / len(df[['id']]) * 100
    print(f"\nPercentage of matching IDs in {name}: {percent_match:.2f}%")


### Venn Diagram 

In [ ]:
import matplotlib.pyplot as plt
from matplotlib_venn import venn3

# Count length of each DB
A, B, C = len(df_2801), len(df_283), len(df_286)
AB, AC, BC, ABC = len(df_2801_intr_283), len(df_2801_intr_286), len(df_283_intr_286), len(df_all)

# Create the Venn diagram 
plt.figure(figsize=(8, 8))
venn_diagram = venn3(subsets=(A, B, AB, C, AC, BC, ABC), set_labels=('KG2.8.0.1', 'KG2.8.3', 'KG2.8.6'))

# Venn Diagram region lables
labels = {'100': A - AB - AC + ABC, '010': B - AB - BC + ABC, '001': C - AC - BC + ABC,
          '110': AB - ABC, '101': AC - ABC, '011': BC - ABC, '111': ABC}

for label, count in labels.items():
    venn_diagram.get_label_by_id(label).set_text(count)

plt.title("Comparison by Node")
plt.show()

**Note:**
The current venn diagramp package for Matplot cannot create proper diagram for certain "fringe" cases.
We recommend drawing the three way venn diagram mannually or use of other illustrator tool to generate the figures.